In [1]:
# !pip install polars numpy implicit tqdm

In [2]:
# !pip install --upgrade pip

In [1]:
# !pip install pyarrow

In [16]:
# !pip install pandas

In [3]:
import polars as pl
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit
import pyarrow
import pandas as pd

RANDOM_STATE = 42
N_PREDICTIONS = 100


In [10]:
!ls

hh_recsys_sample.pq               hh_recsys_train_hh.pq
hh_recsys_solution_example.ipynb  hh_recsys_vacancies.pq
hh_recsys_solution_example_.ipynb venv
hh_recsys_test_hh.pq


In [6]:
train = pl.read_parquet("../data/hh_recsys_train_hh.pq")
test = pl.read_parquet("../data/hh_recsys_test_hh.pq")

In [7]:
vacancies = pl.read_parquet("../data/hh_recsys_vacancies.pq")

In [0]:
# train = train.to_pandas()

In [13]:
# test = test.to_pandas()

In [14]:
# vacancies = vacancies.to_pandas()

In [4]:
# vacancies.to_pickle('vacancies.pkl')

In [18]:
test

,user_id,session_id,vacancy_id,action_type,action_dt
0,u_482520,s_25018731,"[v_2597196, v_1223061, v_1223061]","[2, 2, 1]","[2023-11-19T12:03:13.089000000, 2023-11-19T12:..."
1,u_582132,s_481216,"[v_470400, v_470400, v_1530783]","[2, 1, 2]","[2023-11-21T15:39:47.981000000, 2023-11-21T15:..."
2,u_212584,s_16918781,"[v_1572055, v_1572055, v_1572055, v_953153, v_...","[2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1]","[2023-11-16T08:41:47.031000000, 2023-11-16T08:..."
3,u_425177,s_17505104,"[v_1375331, v_1922852]","[2, 2]","[2023-11-17T12:42:18.513000000, 2023-11-17T12:..."
4,u_700997,s_15528830,"[v_2152997, v_2152997, v_1217630]","[2, 1, 2]","[2023-11-16T17:22:53.530000000, 2023-11-16T17:..."
...,...,...,...,...,...
83184,u_499368,s_19158589,"[v_1716634, v_2232327, v_2232327, v_2232327, v...","[2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2]","[2023-11-18T03:15:15.530000000, 2023-11-18T03:..."
83185,u_1084868,s_24756162,"[v_395706, v_2405231]","[2, 2]","[2023-11-15T18:06:16.550000000, 2023-11-15T18:..."
83186,u_1138032,s_25291467,"[v_1213925, v_1213925, v_2359832, v_2359832]","[2, 1, 2, 1]","[2023-11-19T23:19:44.092000000, 2023-11-19T23:..."
83187,u_608102,s_18813020,"[v_933773, v_933773, v_2021127, v_1508520, v_2...","[3, 2, 2, 2, 2, 1, 2]","[2023-11-16T15:40:09.408000000, 2023-11-16T15:..."


In [19]:
vacancies

,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
0,v_862116,Смотритель музейный,c_162972,<strong>Обязанности:</strong> <ul> <li>Осущест...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"<p><strong>Возможно, наша вакансия именно та, ...","[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,<strong>Обязанности:</strong> <ul> <li>Расчет ...,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,"<p><strong>Для каждого, кто хочет работать и з...",None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience
4,v_312507,Торговый представитель (г. Абакан),c_206699,<p>Компания ТД &quot;Оливьера&quot;- лучший ди...,"[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734124,v_2484959,Сборщик-упаковщик,c_203256,<p><strong>Обязанности:</strong></p> <ul> <li>...,"[Пользователь ПК, Умение работать в команде, У...",40000.0,90000.0,RUR,a_5387,ar_71,full,flexible,noExperience
2734125,v_205163,Сварщик на полуавтомат,c_158695,<p>ОКЛАД 80 000 РУБЛЕЙ. В связи с увеличением ...,"[Желание работать и зарабатывать, Высокая энер...",80000.0,130000.0,RUR,a_5527,ar_69,full,fullDay,between1And3
2734126,v_639897,Главный инженер / Технический директор,c_209365,<p>Компания<strong> МОБИЛЬНЫЙ ДОМ - ЛИДЕР В СФ...,"[Контроль исправности оборудования, Инженерные...",200000.0,NaN,RUR,a_1756,ar_41,full,fullDay,between1And3
2734127,v_1636531,"Провизор/Фармацевт (г.Адыгейск, 20 км от Красн...",c_246244,<strong>Требования:</strong> <ul> <li> <p>Зако...,"[Предпечатная подготовка, Статистический анализ]",25000.0,NaN,RUR,a_3403,ar_60,full,fullDay,noExperience


In [ ]:
vacancies

In [9]:
# Берется массив просмотренных вакансий из теста
# преобразуют их список уникальных вакансий и берут последние 2
# массив из списков преобразуют в список
predictions = test.select(pl.col('vacancy_id').list.unique(maintain_order=True).list.tail(2))['vacancy_id'].to_list()

In [10]:
# predictions

In [11]:
test

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]"


In [ ]:
test = test.with_columns(pl.lit(predictions).alias('predictions'))

In [1]:
print()

In [ ]:
test.with_columns(pl.lit(predictions).alias('predictions'))

In [23]:
test.select(['user_id', 'session_id', 'predictions']).write_parquet('submission.pq')

/tmp/ipykernel_583187/2589823886.py:2: DeprecationWarning: Behavior for `lit` will change for sequence inputs. The result will change to be a literal of type List. To retain the old behavior, pass a Series instead, e.g. `Series(sequence)`.
  test = test.with_columns(pl.lit(predictions).alias('predictions'))


In [24]:
test

user_id,session_id,vacancy_id,action_type,action_dt,predictions
str,str,list[str],list[i64],list[datetime[ns]],list[str]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]","[""v_2597196"", ""v_1223061""]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]","[""v_470400"", ""v_1530783""]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]","[""v_1592343"", ""v_1329494""]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]","[""v_1375331"", ""v_1922852""]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]","[""v_2152997"", ""v_1217630""]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]","[""v_698513"", ""v_212060""]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]","[""v_1828011"", ""v_2297339""]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]","[""v_206700"", ""v_528671""]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]","[""v_36093"", ""v_1500697""]"


# Simple ALS

In [9]:
train.head()

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_332060""","""s_28301374""","[""v_2571684"", ""v_488179"", … ""v_2633899""]","[2, 2, … 2]","[2023-11-01 00:40:58.105, 2023-11-01 00:58:13.091, … 2023-11-01 01:35:54.456]"
"""u_1057881""","""s_33868982""","[""v_665861""]",[2],[2023-11-01 00:23:51.452]
"""u_1036784""","""s_32474802""","[""v_2594840""]",[2],[2023-11-01 00:52:34.023]
"""u_786220""","""s_14060785""","[""v_1473781"", ""v_1622905"", … ""v_2394151""]","[1, 2, … 2]","[2023-11-01 00:58:20.793, 2023-11-01 01:17:04.774, … 2023-11-01 01:17:40.149]"
"""u_639152""","""s_23205986""","[""v_695738"", ""v_22433"", … ""v_1078110""]","[2, 1, … 2]","[2023-11-01 01:14:20.828, 2023-11-01 00:54:08.636, … 2023-11-01 01:00:53.660]"


In [8]:
# Берем уникальных пользователей из train
unique_users = train['user_id'].unique().to_list()
# Берем уникальные вакансии
unique_vacancies = train['vacancy_id'].explode().unique().to_list()

In [10]:
# Создаем словарь {user_id:idx} где user_id идишник пользователя , а idx его порядковый индекс
user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}
# Создаем словарь {vac_id:idx} где vac_id идишник вакансии , а idx ее порядковый индекс
vac2idx = {vac_id: idx for idx, vac_id in enumerate(unique_vacancies)}
# Обратный словарь {idx:vac_id}, где ключ idx
idx2vac = {idx: vac_id for vac_id, idx in vac2idx.items()}
# Создаем словарь с весами по кодам действий
action_weights = {
    1: 4.0, # отклик
    2: 1.0, # просмотр
    3: 2.0 # добавление в избранное
}


In [11]:
# Создаем таблицу pairs с полями user_id, vacancy_id, action_type
pairs = train.select(['user_id', 'vacancy_id', 'action_type']).explode(('vacancy_id', 'action_type'))

In [15]:
# Создаем массив users в котором, подтягиваются порядковые индексы по user_id из словаря user2idx
users = pairs['user_id'].map_dict(user2idx, default=None).to_numpy()

/var/folders/jd/scg9w9y14jngl6m39nfy27k40000gn/T/ipykernel_48030/2855629468.py:1: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  users = pairs['user_id'].map_dict(user2idx, default=None).to_numpy()


In [17]:
# Создаем массив vacancies в котором, подтягиваются порядковые индексы по vac_id из словаря vac2idx
vacancies = pairs['vacancy_id'].map_dict(vac2idx).to_numpy()

/var/folders/jd/scg9w9y14jngl6m39nfy27k40000gn/T/ipykernel_48030/3587250435.py:2: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  vacancies = pairs['vacancy_id'].map_dict(vac2idx).to_numpy()


In [18]:
# Создаем массив preferences в котором, подтягиваются новые веса по коду действия из словаря action_weights
preferences = pairs['action_type'].map_dict(action_weights).to_numpy()

/var/folders/jd/scg9w9y14jngl6m39nfy27k40000gn/T/ipykernel_48030/2422162151.py:2: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  preferences = pairs['action_type'].map_dict(action_weights).to_numpy()


In [21]:
#Создаем sparse матрицу user-vacancy
uv_mat = csr_matrix((preferences, (users, vacancies)))

In [23]:
# uv_mat

In [24]:
#Запускаем ALS
als_model = implicit.als.AlternatingLeastSquares(
    factors=150,
    random_state=RANDOM_STATE,
    iterations=20,
    calculate_training_loss=True,
    regularization=0.001
)
als_model.fit(uv_mat)

/Users/alex/Developer/RecsysHH/venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

In [26]:
# Матрицы с фичами пользователя и вакансий
als_model.user_factors.shape, als_model.item_factors.shape

((882409, 150), (1458353, 150))

In [34]:
# Берем тестовых юзеров в список c user_id
test_users = test['user_id'].to_list()
# Берем тестовые вакансии в список со списками вакансий сохраняя порядок
test_vacancies = test.select(pl.col('vacancy_id').list.unique(maintain_order=True))['vacancy_id'].to_list()

In [ ]:
predictions = []
# Запускаем цикл по парам user_id, vacancy_id

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
    # Если тестовый юзер отсутствует в трэйне, добавляем в predictions все его вакансии из теста
    if user not in user2idx:
        predictions.append(vacs)
        continue
    # определяем индекс текущего пользователя
    cuser = user2idx[user]
    # за
    recommendations = als_model.recommend(cuser, uv_mat[cuser], N=N_PREDICTIONS)[0]
    recommendations = [idx2vac[cv] for cv in recommendations]
    predictions.append(recommendations)

 11%|█▏        | 9382/83189 [07:14<54:37, 22.52it/s]  

In [31]:
test = test.with_columns(pl.lit(predictions).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('als_submission.pq')

/tmp/ipykernel_583187/2455226998.py:1: DeprecationWarning: Behavior for `lit` will change for sequence inputs. The result will change to be a literal of type List. To retain the old behavior, pass a Series instead, e.g. `Series(sequence)`.
  test = test.with_columns(pl.lit(predictions).alias('predictions'))


In [32]:
test

user_id,session_id,vacancy_id,action_type,action_dt,predictions
str,str,list[str],list[i64],list[datetime[ns]],list[str]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]","[""v_2650515"", ""v_367301"", … ""v_2337447""]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]","[""v_1347362"", ""v_1798396"", … ""v_726002""]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]","[""v_1572055"", ""v_953153"", … ""v_1329494""]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]","[""v_2650515"", ""v_367301"", … ""v_676801""]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]","[""v_1431178"", ""v_2395102"", … ""v_1352210""]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]","[""v_1465910"", ""v_1586766"", … ""v_668735""]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]","[""v_701974"", ""v_1081658"", … ""v_27909""]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]","[""v_964765"", ""v_460169"", … ""v_9584""]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]","[""v_1431178"", ""v_2395102"", … ""v_2192945""]"
